In [1]:
from scipy import stats
from xgrads import *
import numpy as np  # 调用numpy
import xarray as xr
from math import *
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

In [2]:
def createmap():  ###############################################生成地图##########################################################
    box = [-180, 180, -90, 90]  # 经度维度
    scale = '110m'  # 地图分辨率
    xstep = 20  # 下面标注经纬度的步长
    ystep = 10
    proj = ccrs.PlateCarree(central_longitude=180)  # 确定地图投影
    fig = plt.figure(figsize=(9, 6))  # dpi=150)###生成底图
    ax = fig.subplots(1, 1, subplot_kw={'projection': proj})  # 确定子图，与grads的类似
    ##海岸线
    ax.coastlines(scale)
    # 标注坐标轴
    ax.set_xticks(np.arange(box[0], box[1] + xstep, xstep), crs=ccrs.PlateCarree())
    ax.set_yticks(np.arange(box[2], box[3] + ystep, ystep), crs=ccrs.PlateCarree())
    # 经纬度格式，把0经度设置不加E和W
    lon_formatter = LongitudeFormatter(zero_direction_label=False)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    ############################################################################################################
    return ax, fig

In [2]:
############NCEP_Z200_30y_Wt转换成nc文件############
hgt = open_CtlDataset('D:\\grads\\TongJi\\NCEP_Z200_30y_Wt.ctl')
hgt.attrs['pdef'] = 'None'
hgt.to_netcdf('D:\\grads\\TongJi\\NCEP_Z200_30y_Wt.nc')
hgt = xr.open_dataset('D:\\grads\\TongJi\\NCEP_Z200_30y_Wt.nc')
############NCEP_TPSST_30y_Wt转换成nc文件###########
air = open_CtlDataset('D:\\grads\\TongJi\\NCEP_TPSST_30y_Wt.ctl')
air.attrs['pdef'] = 'None'
air.to_netcdf('D:\\grads\\TongJi\\NCEP_TPSST_30y_Wt.nc')
air = xr.open_dataset('D:\\grads\\TongJi\\NCEP_TPSST_30y_Wt.nc')
###########NCEP_IOSST_30y_Wt转换成nc文件###########
airindia = open_CtlDataset('D:\\grads\\TongJi\\NCEP_IOSST_30y_Wt.ctl')
airindia.attrs['pdef'] = 'None'
airindia.to_netcdf('D:\\grads\\TongJi\\NCEP_IOSST_30y_Wt.nc')
airindia = xr.open_dataset('D:\\grads\\TongJi\\NCEP_IOSST_30y_Wt.nc')

# 热带太平洋海温

In [4]:
stTP = air['st'][:, :, :]
stIO = airindia['st'][:, :, :]
hgt = hgt['hgt'][:, :, :]
Time = hgt['time'][:]
lon = hgt['lon'][:]
lat = hgt['lat'][:]

In [5]:
stTP = stTP.where(stTP < 99999, np.NAN)
stIO = stIO.where(stIO < 99999, np.NAN)

In [6]:
TPshijian = stTP.mean(dim='lon', skipna=True).mean(dim='lat', skipna=True)
IOshijian = stIO.mean(dim='lon', skipna=True).mean(dim='lat', skipna=True)

In [7]:
avehgt = hgt.mean(dim='time')
aveTP = TPshijian.mean()
aveIO = IOshijian.mean()

In [8]:
def yiyuanhuigui(temp, ave, aveh, h, t):
    tempxy = np.zeros((len(lat), len(lon)))
    tempx2 = 0
    for t_i in range(len(t)):
        tempxy = tempxy + temp[t_i] * h[t_i, :, :]
        tempx2 = tempx2 + temp[t_i] * temp[t_i]
    b = (tempxy - len(t) * ave * aveh) / (tempx2 - len(t) * pow(ave, 2))
    return b


def yiyuanb0(b, ave, aveh):
    b0 = aveh - b * ave
    return b0

In [9]:
lon1, lat1 = np.meshgrid(lon, lat)

In [10]:
##热带太平洋
b = yiyuanhuigui(TPshijian, aveTP, avehgt, hgt, Time)
ax, fig = createmap()
colorbar = ax.contourf(lon1, lat1, b, cmap='bwr', transform=ccrs.PlateCarree())
plt.colorbar(colorbar, extendrect='True', pad=0.03, fraction=0.04, shrink=1)
ax.set_title('Z200andTPSST')
plt.tight_layout()
plt.savefig(r'D:\grads\TongJi\ex4\Z200andTPSST.png')
plt.close()
b0 = yiyuanb0(b, aveTP, avehgt)
ax, fig = createmap()
colorbar = ax.contourf(lon1, lat1, b0, cmap='bwr', transform=ccrs.PlateCarree())
plt.colorbar(colorbar, extendrect='True', pad=0.03, fraction=0.04, shrink=1)
ax.set_title('Z200andTPSSTb0')
plt.tight_layout()
plt.savefig(r'D:\grads\TongJi\ex4\Z200andTPSSTb0.png')
plt.close()

In [11]:
##热带印度洋
b = yiyuanhuigui(IOshijian, aveIO, avehgt, hgt, Time)
ax, fig = createmap()
colorbar = ax.contourf(lon1, lat1, b, cmap='bwr', transform=ccrs.PlateCarree())
plt.colorbar(colorbar, extendrect='True', pad=0.03, fraction=0.04, shrink=1)
ax.set_title('Z200andIOSST')
plt.tight_layout()
plt.savefig(r'D:\grads\TongJi\ex4\Z200andIOSST.png')
plt.close()
b0 = yiyuanb0(b, aveIO, avehgt)
ax, fig = createmap()
colorbar = ax.contourf(lon1, lat1, b0, cmap='bwr', transform=ccrs.PlateCarree())
plt.colorbar(colorbar, extendrect='True', pad=0.03, fraction=0.04, shrink=1)
ax.set_title('Z200andIOSSTb0')
plt.tight_layout()
plt.savefig(r'D:\grads\TongJi\ex4\Z200andIOSSTb0.png')
plt.close()

In [12]:
a1 = np.array(TPshijian).reshape((30, 1))
a2 = np.array(IOshijian).reshape((30, 1))

In [13]:
##多元回归
a = np.full((len(Time), 1), 1)
X = np.hstack((a, a1, a2))
X1 = np.linalg.inv((X.T).dot(X)).dot(X.T)
b = np.zeros((3, len(lat), len(lon)))
for i in range(len(lon)):
    for j in range(len(lat)):
        b[:, j, i] = X1.dot(hgt[:, j, i])

In [14]:
##b1热带太平洋
ax, fig = createmap()
colorbar = ax.contourf(lon1, lat1, b[1, :, :], cmap='bwr', transform=ccrs.PlateCarree())
plt.colorbar(colorbar, extendrect='True', pad=0.03, fraction=0.04, shrink=1)
ax.set_title('Z200andTPSSTb1')
plt.tight_layout()
plt.savefig(r'D:\grads\TongJi\ex4\Z200andTPSSTb1.png')
plt.close()

In [15]:
##b2热带印度洋
ax, fig = createmap()
colorbar = ax.contourf(lon1, lat1, b[2, :, :], cmap='bwr', transform=ccrs.PlateCarree())
plt.colorbar(colorbar, extendrect='True', pad=0.03, fraction=0.04, shrink=1)
ax.set_title('Z200andIOSSTb2')
plt.tight_layout()
plt.savefig(r'D:\grads\TongJi\ex4\Z200andIOSSTb2.png')
plt.close()

In [16]:
##b0
ax, fig = createmap()
colorbar = ax.contourf(lon1, lat1, b[0, :, :], cmap='bwr', transform=ccrs.PlateCarree())
plt.colorbar(colorbar, extendrect='True', pad=0.03, fraction=0.04, shrink=1)
ax.set_title('b0')
plt.tight_layout()
plt.savefig(r'D:\grads\TongJi\ex4\b0.png')
plt.close()

In [17]:
#偏相关.
def xiang_guan_xi_shu(ave, aveh, shijian, shijian2, lenT):
    if aveh.size == 1:
        temp1 = 0
        temp2 = 0
        temp3 = 0
        for t_i in range(len(lenT)):
            temp1 = temp1 + (shijian[t_i] - ave) * (shijian2[t_i] - aveh)
            temp2 = temp2 + pow(shijian[t_i] - ave, 2)
            temp3 = temp3 + pow(shijian2[t_i] - aveh, 2)
        temp1 = temp1 / len(lenT)
        temp2 = sqrt(temp2 / len(lenT))
        temp3 = sqrt(temp3 / len(lenT))
    else:
        temp1 = np.zeros((len(lat), len(lon)))
        temp2 = 0
        temp3 = np.zeros((len(lat), len(lon)))
        for t_i in range(len(lenT)):
            temp1 = temp1 + (shijian[t_i] - ave) * (shijian2[t_i, :, :] - aveh[:, :])
            temp2 = temp2 + pow(shijian[t_i] - ave, 2)
            temp3 = temp3 + (shijian2[t_i, :, :] - aveh[:, :]) * (shijian2[t_i, :, :] - aveh[:, :])
        temp1 = temp1 / len(lenT)
        temp2 = sqrt(temp2 / len(lenT))
        temp3 = np.sqrt(temp3 / len(lenT))
    R = temp1 / (temp2 * temp3)

    return R

In [18]:
r12 = xiang_guan_xi_shu(aveTP, aveIO, TPshijian, IOshijian, Time)
r1y = xiang_guan_xi_shu(aveTP, avehgt, TPshijian, hgt, Time)
r2y = xiang_guan_xi_shu(aveIO, avehgt, IOshijian, hgt, Time)

In [19]:
ry12 = (r1y - r12 * r2y) / np.sqrt((1 - r2y * r2y) * (1 - r12 * r12))
ry21 = (r2y - r12 * r1y) / np.sqrt((1 - r1y * r1y) * (1 - r12 * r12))

In [20]:
##ry12
ax, fig = createmap()
colorbar = ax.contourf(lon1, lat1, ry12, cmap='bwr', transform=ccrs.PlateCarree())
plt.colorbar(colorbar, extendrect='True', pad=0.03, fraction=0.04, shrink=1)
ax.set_title('ry12')
plt.tight_layout()
plt.savefig(r'D:\grads\TongJi\ex4\Ry12.png')
plt.close()

In [21]:
#ry21
ax, fig = createmap()
colorbar = ax.contourf(lon1, lat1, ry21, cmap='bwr', transform=ccrs.PlateCarree())
plt.colorbar(colorbar, extendrect='True', pad=0.03, fraction=0.04, shrink=1)
ax.set_title('ry21')
plt.tight_layout()
plt.savefig(r'D:\grads\TongJi\ex4\Ry21.png')
plt.close()